## Vertical Model

In [60]:
import pyblp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import copy 
import statsmodels.api as sm

pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

'0.12.0'

In [86]:
dataset= pd.read_csv('Yr18_PSet_BLP_data_no_header.txt', delimiter= '\t',
                     header=None, names= ['prices', 'quantity', 'weight', 'power', 'ac', 'firm_ids'])
dataset['market_ids']=1
dataset['prices']= dataset['prices']/10000
dataset['weight']= dataset['weight']/1000
dataset['power']= dataset['power']/100
M= 100000000 # Market size: 100 million
dataset['shares']= dataset['quantity']/M
dataset[['shares', 'prices', 'quantity', 'weight', 'power', 'ac']].describe()



,shares,prices,quantity,weight,power,ac
count,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000
mean,0.000664,1.834909,66377.312977,2.923466,1.335878,0.458015
std,0.000749,1.234279,74854.734184,0.564956,0.458746,0.500147
min,0.000010,0.443500,1037.000000,1.620000,0.550000,0.000000
25%,0.000145,1.041250,14488.500000,2.526500,1.000000,0.000000
50%,0.000396,1.380000,39602.000000,2.839000,1.300000,0.000000
75%,0.000948,2.340000,94776.000000,3.329000,1.600000,1.000000
max,0.004172,7.380000,417179.000000,4.283000,2.780000,1.000000


In [105]:
# Some global variables
J= len(dataset.index)
share_outside= 1-sum(dataset.shares)
dataset= dataset.sort_values('prices', ignore_index=True)
dataset

,prices,quantity,weight,power,ac,firm_ids,market_ids,shares
0,0.4435,6359,1.870,0.67,0,23,1,0.000064
1,0.5866,14363,1.820,0.73,0,5,1,0.000144
2,0.5899,100590,2.040,0.81,0,21,1,0.001006
3,0.5909,4665,2.336,0.76,0,10,1,0.000047
4,0.5995,52409,1.620,0.55,0,19,1,0.000524
...,...,...,...,...,...,...,...,...
126,4.9000,10460,3.835,2.08,1,8,1,0.000105
127,5.3050,3407,3.446,2.00,1,19,1,0.000034
128,5.8500,3650,3.031,2.47,1,12,1,0.000036
129,6.2500,6715,3.915,2.01,1,9,1,0.000067


In [90]:
# Construct instruments:
# ac is dummy, so perhaps not as helpful
blp_instruments = pyblp.build_blp_instruments(pyblp.Formulation('1 + weight + power '), dataset)
gh_instruments = pyblp.build_blp_instruments(pyblp.Formulation(' weight + power '), dataset)

#### Question 1
For i=1$\rightarrow $J:

- For good 1, the outside option's share is $s_{0}\rightarrow \alpha
_{1}=-\frac{\ln (s_{0})}{\lambda }.$ Hence we can solve the value of $\delta
_{1}~$from the equation: $\delta _{1}=\alpha _{1}p_{1}$ $(\alpha
_{N+1}=0,\alpha _{0}=\infty ).$

- For good i, we can then solve its $\delta _{i}$ recursively with the
following steps:

    * Step1: From market share, we pin down $\left\{ \alpha _{i}\right\} $
recursively: $e^{-\lambda \alpha _{i+1}}-e^{-\lambda \alpha _{i}}=s_{i}.$ $%
\Rightarrow \alpha _{i}=-\frac{\ln (e^{-\lambda \alpha _{i-1}}+s_{i-1})}{%
\lambda }.$

    * Step2: With price and $\alpha $ we pin down $\delta $: $\delta
_{i}=\delta _{i-1}+\alpha _{i}p_{i}-\alpha _{i}p_{i-1}.$ Hence we can get $%
\delta _{i}.$

- Finally we have all $\delta _{i}.$ Note that $\delta _{i}=\xi
_{i}+\beta X.$ This is a OLS question. We can solve it using a standard
method. Or if we believe that $\xi _{j}$ is correlated with $p_{j}$ , we
need to find some instrument to run the regression.


In [110]:

lamda= 1/(4*10**(-6)*10**4);

alpha= np.zeros([J,1])
delta= np.zeros([J,1])
alpha[0,0]= -np.log(share_outside)/lamda;
delta[0,0]= alpha[0,0]*dataset['prices'][0]; 
# horizontal normalization, outside option brings utility of 0.
for i in range(J-1):
    alpha[i+1,0]=-np.log(np.exp(-lamda*alpha[i,0]) +dataset['shares'][i])/lamda;
    delta[i+1,0]= delta[i,0]- alpha[i+1,0]*dataset['prices'][i]+ alpha[i+1,0]*dataset['prices'][i+1];


In [111]:
X= np.hstack((np.ones([J,1]), dataset[['weight', 'power', 'ac']].values))
dataset['constant']=1.

In [192]:
model = sm.OLS(delta, dataset[['constant','weight', 'power', 'ac']])
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.736
Model:                            OLS   Adj. R-squared:                  0.730
Method:                 Least Squares   F-statistic:                     118.0
Date:                Thu, 08 Apr 2021   Prob (F-statistic):           1.46e-36
Time:                        23:56:02   Log-Likelihood:                 838.94
No. Observations:                 131   AIC:                            -1670.
Df Residuals:                     127   BIC:                            -1658.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
constant       0.0015      0.000      5.669      0.0

In [114]:
import statsmodels.sandbox.regression.gmm as gmm
model = gmm.IVGMM(delta, dataset[['constant','weight', 'power', 'ac']], 
                  np.hstack((dataset[['constant','weight', 'power', 'ac']].values, gh_instruments)))
results_iv = model.fit()
print(results_iv.summary())

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 11
         Current function value: 0.036778
         Iterations: 5
         Function evaluations: 53
         Gradient evaluations: 43
Optimization terminated successfully.
         Current function value: 0.037318
         Iterations: 5
         Function evaluations: 9
         Gradient evaluations: 9
Optimization terminated successfully.
         Current function value: 0.037353
         Iterations: 5
         Function evaluations: 7
         Gradient evaluations: 7
Optimization terminated successfully.
         Current function value: 0.037354
         Iterations: 5
         Function evaluations: 8
         Gradient evaluations: 8
                                IVGMM Results                                 
Dep. Variable:                      y   Hansen J:                        4.893
Model:                         

In [116]:
from statsmodels.iolib.summary2 import summary_col
info_dict={'R-squared' : lambda x: f"{x.rsquared:.2f}",
           'No. observations' : lambda x: f"{int(x.nobs):d}"}

results_table = summary_col(results=[results,results_iv],
                            float_format='%0.4f',
                            stars = True,
                            model_names=['OLS',
                                         'IV'],
                            info_dict=info_dict,
                            regressor_order=['const',
                                             'weight',
                                             'power',
                                             'ac'])
results_table.add_title('Table 2 - OLS Regressions')

print(results_table)

     Table 2 - OLS Regressions
                    OLS        IV   
------------------------------------
weight           0.0005*** 0.0006***
                 (0.0001)  (0.0001) 
power            0.0005*** 0.0006***
                 (0.0001)  (0.0002) 
ac               0.0003*** 0.0003*  
                 (0.0001)  (0.0002) 
constant         0.0015*** 0.0013***
                 (0.0003)  (0.0005) 
R-squared        0.7360             
R-squared Adj.   0.7298             
R-squared        0.74               
No. observations 131       131      
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


#### Question 2:
Price issue: if there are two products with same price, they will have the same slope, hence they must
have a same intercept (otherwise one of them is strictly dominated and will have a share of 0). So we
can only match their total share. In other word, we cannot seperately identify their $\delta_j$ :

#### Question 3: Supply side

- How to include the information from supply side? We can start our
trial from profit maximization problem:

$
\begin{split}
&\max_{\{p_{i}\}_{i\in a}}\sum_{i\in a}p_{i}(q_{i})q_{i}-c(q_{i}) \\
&F.O.C\Rightarrow ~(p_{j}-mc_{j}(q_{j}))\sum_{i\in a}\frac{\partial q_{j}}{%
\partial p_{i}}+q_{j}=0 \\
&\Rightarrow ~p_{j}-(\frac{\partial q}{\partial p})^{-1}q_{j}=mc(q_{i}) \\
&\text{In matrix form,}~p-\Delta ^{-1}s=mc,\Delta =\Omega \ast (\frac{%
\partial s}{\partial p})
\end{split}
$

note that $\left( \frac{\partial s}{\partial p}\right) ^{-1}=\left( \frac{%
\partial q/Q}{\partial p}\right) ^{-1}\times q/Q$ and we the specification
of $mc(q_{i})=x_{j}\gamma +\eta q_{j}+w_{j}.$

Clearly, what would possibly affect $q_{i}$ is the price of nearby products,
i+1 and i-1. If these products happen to be controlled by the same producer,
they can actually maximize total profit and hence have a higher profit.

1. **MC**: If the strategy is simply marginal price= marginal cost, then: $%
p_{j}=x_{j}\gamma +\eta q_{j}+w_{j}.$

2. **CP**: Start form the easy case that it is a competitive market. Note that $(%
\frac{\partial q_{i}}{\partial p_{j}})^{-1}q_{i}=(\frac{\partial q_{i}}{%
\partial p_{j}})^{-1}S_{i}.$

$\delta _{i}=\delta _{i-1}+\alpha _{i}p_{i}-\alpha _{i}p_{i-1}.(\alpha _{i}=%
\frac{\delta _{i}-\delta _{i-1}}{p_{i}-p_{i-1}})$ and $\delta _{i+1}=\delta
_{i}+\alpha _{i+1}p_{i+1}-\alpha _{i+1}p_{i}.(\alpha _{i+1}=\frac{\delta
_{i+1}-\delta _{i}}{p_{i+1}-p_{i}})$

Share is given by $s_{i}=e^{-\lambda \alpha _{i+1}}-e^{-\lambda \alpha
_{i}}. $%
$
\begin{split}
\frac{\partial s}{\partial p}_{i,i} &=&\frac{\partial s_{i}}{\partial p_{i}}=%
\frac{\partial s_{i}}{\partial \alpha _{i+1}}\frac{\partial \alpha _{i+1}}{%
\partial p_{i}}+\frac{\partial s_{i}}{\partial \alpha _{i}}\frac{\partial
\alpha _{i}}{\partial p_{i}}=-\lambda e^{-\lambda \alpha _{i+1}}\frac{\delta
_{i+1}-\delta _{i}}{(p_{i+1}-p_{i})^{2}}-\lambda e^{-\lambda \alpha _{i}}%
\frac{\delta _{i}-\delta _{i-1}}{(p_{i}-p_{i-1})^{2}}. \\
\frac{\partial s}{\partial p}_{i,i+1} &=&\frac{\partial s_{i}}{\partial
p_{i+1}}=\frac{\partial s_{i}}{\partial \alpha _{i+1}}\frac{\partial \alpha
_{i+1}}{\partial p_{i+1}}+\frac{\partial s_{i}}{\partial \alpha _{i}}\frac{%
\partial \alpha _{i}}{\partial p_{i+1}}=\lambda e^{-\lambda \alpha _{i+1}}%
\frac{\delta _{i+1}-\delta _{i}}{(p_{i+1}-p_{i})^{2}}+\lambda e^{-\lambda
\alpha _{i}}\ast 0 \\
&=&\lambda e^{-\lambda \alpha _{i+1}}\frac{\delta _{i+1}-\delta _{i}}{%
(p_{i+1}-p_{i})^{2}}. \\
\frac{\partial s}{\partial p}_{i,i-1} &=&\frac{\partial s_{i}}{\partial
p_{i-1}}=\frac{\partial s_{i}}{\partial \alpha _{i+1}}\frac{\partial \alpha
_{i+1}}{\partial p_{i-1}}+\frac{\partial s_{i}}{\partial \alpha _{i}}\frac{%
\partial \alpha _{i}}{\partial p_{i-1}}=\lambda e^{-\lambda \alpha
_{i+1}}\ast 0+\lambda e^{-\lambda \alpha _{i}}\frac{\delta _{i}-\delta _{i-1}%
}{(p_{i}-p_{i-1})^{2}}. \\
&=&\lambda e^{-\lambda \alpha _{i}}\frac{\delta _{i}-\delta _{i-1}}{%
(p_{i}-p_{i-1})^{2}}.
\end{split}
$

Anything else in matrix $\mathbf{(}\frac{\partial s}{\partial p}\mathbf{)}$
would be zero.

Define the $\Omega $ ownership matrix. In this setting only $\Omega _{ii}=1.$

We can then find that this does not depend on any demand side parameters $%
\beta $. So it follows that we can use OLS or GMM to estimate supply side
parameters.

3. **MP**: simply change $\Omega _{ij}=1$ if i product is owned by j firm.

4. **CL**: simply change $\Omega =\mathbf{1}_{J\ast J}$.

In [133]:
S= np.zeros([J,J])
#DShare/Dprice matrix
price= dataset['prices']
for i in range(J):
    if i==0:
        S[i,i]= -lamda*np.exp(-lamda*alpha[i+1,0]) * (delta[i+1,0]-delta[i,0])/(price[i+1]-price[i])**2 
        -lamda*np.exp(-lamda*alpha[i,0]) * (delta[i,0])/(price[i])**2;
        S[i,i+1]= lamda*np.exp(-lamda*alpha[i+1,0]) * (delta[i+1,0]-delta[i,0])/(price[i+1]-price[i])**2
    if i!=0 and i!=J-1:
        S[i,i]= -lamda*np.exp(-lamda*alpha[i+1,0]) * (delta[i+1,0]-delta[i,0])/(price[i+1]-price[i])**2
        -lamda*np.exp(-lamda*alpha[i,0]) * (delta[i,0]-delta[i-1,0])/(price[i]-price[i-1])**2
        S[i,i+1]= lamda*np.exp(-lamda*alpha[i+1,0]) * (delta[i+1,0]-delta[i,0])/(price[i+1]-price[i])**2
        S[i,i-1]= lamda*np.exp(-lamda*alpha[i,0]) * (delta[i,0]-delta[i-1,0])/(price[i]-price[i-1])**2
    if i==J-1:
        S[i,i]= -lamda*np.exp(-lamda*alpha[i,0]) * (delta[i,0]-delta[i-1,0])/(price[i]-price[i-1])**2
        S[i,i-1]= lamda*np.exp(-lamda*alpha[i,0]) * (delta[i,0]-delta[i-1,0])/(price[i]-price[i-1])**2


In [234]:
# Case1: marginal cost pricing $mc_{j}=p_{j}=x_{j}\gamma+ \eta q_{j} + w_{j}$
Omega1= np.zeros([J,J]) 
# Ownership matrix

# If joint estimate
mc1= price
dep= np.vstack((np.reshape(mc1.values, (-1, 1)), delta))
X1= np.hstack((np.ones([J,1]), dataset[['quantity', 'weight', 'power', 'ac']].values))
X2= np.hstack((np.ones([J,1]), dataset[['weight', 'power', 'ac']].values))
Z= np.hstack((np.ones([J,1]), dataset[['weight', 'power', 'ac']].values, gh_instruments[:,0:2]))
indep= sp.linalg.block_diag(X1, X2)
instrument=np.kron(np.eye(2),Z);
import statsmodels.sandbox.regression.gmm as gmm
model_mc = gmm.IVGMM(dep, indep, instrument)
results_mc = model_mc.fit()
print(results_mc.summary())

# If not joint:
from linearmodels import IV2SLS, IVLIML, IVGMM, IVGMMCUE
mc1= price
model_mc= IVGMM(mc1, dataset[['constant', 'weight', 'power', 'ac']],  dataset['quantity'], gh_instruments[:,0:3])
results_mc = model_mc.fit()
results_mc

Optimization terminated successfully.
         Current function value: 0.041453
         Iterations: 20
         Function evaluations: 25
         Gradient evaluations: 25
Optimization terminated successfully.
         Current function value: 0.045187
         Iterations: 28
         Function evaluations: 32
         Gradient evaluations: 32
Optimization terminated successfully.
         Current function value: 0.037818
         Iterations: 24
         Function evaluations: 28
         Gradient evaluations: 28
Optimization terminated successfully.
         Current function value: 0.037094
         Iterations: 24
         Function evaluations: 27
         Gradient evaluations: 27
Optimization terminated successfully.
         Current function value: 0.036963
         Iterations: 23
         Function evaluations: 28
         Gradient evaluations: 28
         Current function value: 0.036936
         Iterations: 24
         Function evaluations: 84
         Gradient evaluations: 73
Optimi

IV-GMM Estimation Summary                           
==============================================================================
Dep. Variable:                 prices   R-squared:                      0.6789
Estimator:                     IV-GMM   Adj. R-squared:                 0.6688
No. Observations:                 131   F-statistic:                    260.24
Date:                Fri, Apr 09 2021   P-value (F-stat)                0.0000
Time:                        00:32:46   Distribution:                  chi2(4)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
constant      -0.0008     0.4109    -0.0020     0.9984     -0.8062      0.8045
weight        -0.1462     0.1994    -0.7333     0.4634     -0.5371      0.2446
power          1.5382     0.3227     4.7668     0.0000      0.9057      2.1706
ac             0.5479     0.1861     2.9433     0.0032      0.1830      0.9127
quantity   -1.851e-06   1.53e-06    -1.2102     0.2262  -4.849e-06   1.147e-06
==============================================================================

Endogenous: quantity
Instruments: instruments.0, instruments.1, instruments.2
GMM Covariance
Debiased: False
Robust (Heteroskedastic)
IVGMMResults, id: 0x1d0fea1d708

In [180]:
mc2

0      0.443390
1      0.586594
2      0.589888
3      0.590895
4      0.599291
         ...   
126    4.671533
127    5.182323
128    5.722892
129    4.659864
130    6.249973
Name: prices, Length: 131, dtype: float64

In [206]:
# Case2: single product firm
from linearmodels import IV2SLS, IVLIML, IVGMM, IVGMMCUE
Omega2= np.eye(J)
mc2= price+ np.linalg.lstsq(Omega2* S, dataset['shares'].values)[0]
model_sp= IVGMM(mc2, dataset[['constant', 'weight', 'power', 'ac']],  dataset['quantity'], gh_instruments[:,0:3])
results_sp = model_sp.fit()
results_sp

C:\Users\13695\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  after removing the cwd from sys.path.


IV-GMM Estimation Summary                           
==============================================================================
Dep. Variable:                 prices   R-squared:                      0.7307
Estimator:                     IV-GMM   Adj. R-squared:                 0.7222
No. Observations:                 131   F-statistic:                    280.13
Date:                Fri, Apr 09 2021   P-value (F-stat)                0.0000
Time:                        00:03:52   Distribution:                  chi2(4)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
constant      -0.1568     0.3689    -0.4252     0.6707     -0.8798      0.5661
weight        -0.1165     0.1911    -0.6095     0.5422     -0.4910      0.2581
power          1.5903     0.2974     5.3471     0.0000      1.0074      2.1732
ac             0.5112     0.1792     2.8528     0.0043      0.1600      0.8625
quantity   -1.555e-06  1.484e-06    -1.0478     0.2948  -4.465e-06   1.354e-06
==============================================================================

Endogenous: quantity
Instruments: instruments.0, instruments.1, instruments.2
GMM Covariance
Debiased: False
Robust (Heteroskedastic)
IVGMMResults, id: 0x1d0fe98ee88

In [228]:
np.vstack((mc1, mc2, mc3)).T
Omega3[5,5]

1.0

In [230]:
# Case3: multiple product firm
Omega3= np.zeros([J,J])
for i in range(J):
    for k in range(J):
        if dataset['firm_ids'][i]== dataset['firm_ids'][k]:
            Omega3[i,k]=1 
            
mc3= price+ np.linalg.lstsq(Omega3* S, dataset['shares'].values)[0]
model_mp= IVGMM(mc3, dataset[['constant', 'weight', 'power', 'ac']],  dataset['quantity'], gh_instruments[:,0:3])
results_mp = model_sp.fit()
results_mp

C:\Users\13695\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


IV-GMM Estimation Summary                           
==============================================================================
Dep. Variable:                 prices   R-squared:                      0.7307
Estimator:                     IV-GMM   Adj. R-squared:                 0.7222
No. Observations:                 131   F-statistic:                    280.13
Date:                Fri, Apr 09 2021   P-value (F-stat)                0.0000
Time:                        00:30:28   Distribution:                  chi2(4)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
constant      -0.1568     0.3689    -0.4252     0.6707     -0.8798      0.5661
weight        -0.1165     0.1911    -0.6095     0.5422     -0.4910      0.2581
power          1.5903     0.2974     5.3471     0.0000      1.0074      2.1732
ac             0.5112     0.1792     2.8528     0.0043      0.1600      0.8625
quantity   -1.555e-06  1.484e-06    -1.0478     0.2948  -4.465e-06   1.354e-06
==============================================================================

Endogenous: quantity
Instruments: instruments.0, instruments.1, instruments.2
GMM Covariance
Debiased: False
Robust (Heteroskedastic)
IVGMMResults, id: 0x1d0fe9be648

In [231]:
Omega4= np.ones([J,J])        
mc4= price+ np.linalg.lstsq(Omega4* S, dataset['shares'].values)[0]
model_cl= IVGMM(mc4, dataset[['constant', 'weight', 'power', 'ac']],  dataset['quantity'], gh_instruments[:,0:3])
results_cl = model_cl.fit()
results_cl

C:\Users\13695\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


IV-GMM Estimation Summary                           
==============================================================================
Dep. Variable:                 prices   R-squared:                      0.6794
Estimator:                     IV-GMM   Adj. R-squared:                 0.6692
No. Observations:                 131   F-statistic:                    242.73
Date:                Fri, Apr 09 2021   P-value (F-stat)                0.0000
Time:                        00:31:00   Distribution:                  chi2(4)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
constant      -0.0478     0.4109    -0.1162     0.9075     -0.8532      0.7576
weight        -0.1636     0.2128    -0.7690     0.4419     -0.5806      0.2534
power          1.5997     0.3386     4.7247     0.0000      0.9361      2.2633
ac             0.5552     0.1876     2.9589     0.0031      0.1874      0.9230
quantity   -1.525e-06  1.598e-06    -0.9542     0.3400  -4.656e-06   1.607e-06
==============================================================================

Endogenous: quantity
Instruments: instruments.0, instruments.1, instruments.2
GMM Covariance
Debiased: False
Robust (Heteroskedastic)
IVGMMResults, id: 0x1d0fe963d88

In [235]:
from collections import OrderedDict
from linearmodels.iv.results import compare
res = OrderedDict()
res['MC'] = results_mc
res['SP'] = results_sp
res['MP'] = results_mp
res['CL'] = results_cl

print(compare(res))

                                  Model Comparison                                  
                                  MC              SP              MP              CL
------------------------------------------------------------------------------------
Dep. Variable                 prices          prices          prices          prices
Estimator                     IV-GMM          IV-GMM          IV-GMM          IV-GMM
No. Observations                 131             131             131             131
Cov. Est.                     robust          robust          robust          robust
R-squared                     0.6789          0.7307          0.7307          0.6794
Adj. R-squared                0.6688          0.7222          0.7222          0.6692
F-statistic                   260.24          280.13          280.13          242.73
P-value (F-stat)              0.0000          0.0000          0.0000          0.0000
==================      ============    ============    =========